# HW8 Solution Walkthrough (R)
#### Columbia University APAN 5310 - SQL & Relational Databases (Fall 2018)

*Prepared by Nick Machairas. Posted on Nov. 13, 2018*

---

First, import necessary packages:

In [1]:
require('RPostgreSQL')

Loading required package: RPostgreSQL
Loading required package: DBI


## Dataset inspection

Load the ``csv`` file in a dataframe, ``df``:

In [2]:
df <- read.csv('hw8_dataset_f18.csv')

In [3]:
head(df)

first_name,last_name,email,cell_phone,movie_title,movie_genres,purchase_datetime,movie_price
Jackqueline,Dulanty,jdulanty0@goo.ne.jp,+960-657-528-2938,"Thirteenth Floor, The",Drama|Sci-Fi|Thriller,2018-08-27 17:29:17,4.04
Merrili,Fost,mfost1@nps.gov,+351-633-272-4106,"Furies, The",Drama|Romance|Western,2018-08-20 08:42:38,6.96
Shae,Haberfield,shaberfield2@ihg.com,+62-629-240-2964,Smart Money,Crime|Drama,2018-07-24 00:15:35,13.47
Kennie,Church,kchurch3@nature.com,+381-486-928-8686,Wishful Thinking,Comedy,2018-09-22 11:01:17,13.40
Shadow,Puckett,spuckett4@shareasale.com,+27-611-497-3264,You Are God (Jestes Bogiem),Drama,2018-01-15 08:41:31,10.38
Rici,Burnett,rburnett5@state.tx.us,,Through the Forest,Drama,2018-04-01 05:16:02,5.36


Always good practice to check for repeating and null values in the dataset.

In [4]:
summary(df)

   first_name       last_name                          email    
 Chicky :  3   Bare      :  2   aaburrowcl@telegraph.co.uk:  1  
 Lois   :  3   Blackmuir :  2   aackeroydp5@upenn.edu     :  1  
 Sasha  :  3   Breem     :  2   aairdrie8v@hud.gov        :  1  
 Shae   :  3   Brettle   :  2   aajeanon@mac.com          :  1  
 Abbye  :  2   Crossgrove:  2   aarzu7a@addtoany.com      :  1  
 Alfonse:  2   Hamilton  :  2   abannon6b@google.nl       :  1  
 (Other):984   (Other)   :988   (Other)                   :994  
           cell_phone              movie_title          movie_genres
                :110   Whirlygirl        :  3   Drama         :153  
 +1-113-110-7075:  1   Afterwards        :  2   Comedy        :103  
 +1-202-880-6525:  1   Borrowers, The    :  2   Documentary   : 64  
 +1-215-311-2939:  1   Burn After Reading:  2   Drama|Romance : 40  
 +1-239-985-2589:  1   Chosen, The       :  2   Comedy|Drama  : 39  
 +1-286-233-3268:  1   Dirty Dozen, The  :  2   Comedy|Romance: 23

As we can see above, there are 1000 rows in the dataframe. All columns have been assigned "object" datatype except for movie price that was interpreted as "numeric". Also, all columns have data for 1000 rows except for "cell_phone" which has data in 890 rows, hence, 110 NULL values. We will need to take this in to account when designing the database tables. In the context of online orders, an email is perhaps more important than a phone number, hence, it should be acceptable to have NULL values for cell phone numbers.

Next, we have to check for repeating values. Are there 1000 individual movie titles? If not, is the price equal among the same movie titles? Are there 1000 individual customers?

For movie titles, we can see from the output of the summary function above that there are indeed titles which are repeating such as "Whirlygirl" and "Afterwards". What about the prices of these movies?

In [5]:
df[c('movie_title', 'movie_price')][df$movie_title %in% c('Afterwards','Whirlygirl'),]

,movie_title,movie_price
156,Whirlygirl,2.92
504,Afterwards,13.13
525,Afterwards,14.88
695,Whirlygirl,2.24
816,Whirlygirl,17.97


Movie price is different per purchase. This must serve as a hint that prices are not linked to the movies, rather, price is set at checkout. For example, a customer might have a coupon and get a discount on the movie.

Moving on to unique customers...

In [6]:
df[duplicated(df[c('first_name', 'last_name')]),]

first_name,last_name,email,cell_phone,movie_title,movie_genres,purchase_datetime,movie_price


Since the length of the resulting subset of ``df`` by using ``duplicated()`` is 0, we can assert that within the given dataset, there are no orders made by the same person. Keep in mind however, that this might be possible in larger datasets and in any case it is reasonable to assume that one customer can purchase multiple movies.

## Normalization

- Normalization plan can be designed with a few rows of data, not the entire dataset
- Movies can have multiple genres, seperated with the pipe symbol ("|")

Let's work with the first three rows of data of the original, unnormalized dataset:

In [7]:
head(df,3)

first_name,last_name,email,cell_phone,movie_title,movie_genres,purchase_datetime,movie_price
Jackqueline,Dulanty,jdulanty0@goo.ne.jp,+960-657-528-2938,"Thirteenth Floor, The",Drama|Sci-Fi|Thriller,2018-08-27 17:29:17,4.04
Merrili,Fost,mfost1@nps.gov,+351-633-272-4106,"Furies, The",Drama|Romance|Western,2018-08-20 08:42:38,6.96
Shae,Haberfield,shaberfield2@ihg.com,+62-629-240-2964,Smart Money,Crime|Drama,2018-07-24 00:15:35,13.47


### 1NF: Domains of all table attributes must be atomic and there cannot be repeating attributes

- Primary keys are noted with (PK)
- The original ``movie_genres`` column was not atomic and was separated into the ``genres`` table.
- The ``customers_movies`` table has a composite primary key. Rows were expanded to relate movies to genres.


#### ``customer_movies`` table:

|customer_id (PK)|genre_id (PK)|first_name|last_name|email|cell_phone|movie_title|purchase_datetime|movie_price|
|----|----|----|----|----|----|----|----|----|
|1|1|Jackqueline|Dulanty|jdulanty0@goo.ne.jp|+960-657-528-2938|Thirteenth Floor, The|2018-08-27 17:29:17|4.04|
|1|2|Jackqueline|Dulanty|jdulanty0@goo.ne.jp|+960-657-528-2938|Thirteenth Floor, The|2018-08-27 17:29:17|4.04|
|1|3|Jackqueline|Dulanty|jdulanty0@goo.ne.jp|+960-657-528-2938|Thirteenth Floor, The|2018-08-27 17:29:17|4.04|
|2|1|Merrili|Fost|mfost1@nps.gov|+351-633-272-4106|Furies, The|2018-08-20 08:42:38|6.96|
|2|4|Merrili|Fost|mfost1@nps.gov|+351-633-272-4106|Furies, The|2018-08-20 08:42:38|6.96|
|2|5|Merrili|Fost|mfost1@nps.gov|+351-633-272-4106|Furies, The|2018-08-20 08:42:38|6.96|
|3|6|Shae|Haberfield|shaberfield2@ihg.com|+62-629-240-2964|Smart Money|2018-07-24 00:15:35|13.47|
|3|1|Shae|Haberfield|shaberfield2@ihg.com|+62-629-240-2964|Smart Money|2018-07-24 00:15:35|13.47|
|...|...|...|...|...|...|...|...|...|



#### ``genres`` table:

|genre_id (PK)|genre|
|----|----|
|1|Drama|
|2|Sci-Fi|
|3|Thriller|
|4|Romance|
|5|Western|
|6|Crime|
|...|...|




### 2NF: Must be in 1NF and every non-key attribute must be fully dependent on the key

- ``first_name``, ``last_name``, ``email`` and ``cell_phone`` were moved to a new ``customers`` table to satisfy 2NF rules
- ``movie_title`` and ``movie_price`` were moved to a new ``movies`` table to satisfy 2NF rules.
- a new ``movie_genres`` table was created to remove the ``genre_id`` from the 1NF ``customers_movies`` table and satisfy 2NF rules. Notice that this way movies can have multiple genres and if the name of a genre is updated it will reflect on all movies.
- a new ``orders`` table was created to accomodate the ``purchase_datetime`` attribute and relate movies to customers


#### ``customer`` table:

|customer_id (PK)|first_name|last_name|email|cell_phone|
|----|----|----|----|----|
|1|Jackqueline|Dulanty|jdulanty0@goo.ne.jp|+960-657-528-2938|
|2|Merrili|Fost|mfost1@nps.gov|+351-633-272-4106|
|3|Shae|Haberfield|shaberfield2@ihg.com|+62-629-240-2964|
|...|...|...|...|...|


#### ``movies`` table:

|movie_id (PK)|movie_title|movie_price|
|----|----|----|
|1|Thirteenth Floor, The|4.04|
|2|Furies, The|6.96|
|3|Smart Money|13.47|
|...|...|...|


#### ``genres`` table:

|genre_id (PK)|genre|
|----|----|
|1|Drama|
|2|Sci-Fi|
|3|Thriller|
|4|Romance|
|5|Western|
|6|Crime|
|...|...|


#### ``movie_genres`` table:

|movie_id (PK)|genre_id (PK)|
|----|----|
|1|1|
|1|2|
|1|3|
|2|1|
|2|4|
|2|5|
|3|6|
|3|1|
|...|...|

#### ``orders`` table:

|customer_id (PK)|movie_id (PK)|purchase_datetime|
|----|----|----|
|1|1|2018-08-27 17:29:17|
|2|2|2018-08-20 08:42:38|
|3|3|2018-07-24 00:15:35|
|...|...|...|

### 3NF: Must be in 2NF and every non-key attribute must be non-transitively dependent on the key

This is a little bit tricky... When inspecting the dataset, we observed that prices for the same movie might not be the same. In which case ``movie_price`` should be part of the ``orders`` table instead of the ``movies`` table. 


#### ``customer`` table:

|customer_id (PK)|first_name|last_name|email|cell_phone|
|----|----|----|----|----|
|1|Jackqueline|Dulanty|jdulanty0@goo.ne.jp|+960-657-528-2938|
|2|Merrili|Fost|mfost1@nps.gov|+351-633-272-4106|
|3|Shae|Haberfield|shaberfield2@ihg.com|+62-629-240-2964|
|...|...|...|...|...|


#### ``movies`` table:

|movie_id (PK)|movie_title|
|----|----|
|1|Thirteenth Floor, The|
|2|Furies, The|
|3|Smart Money|
|...|...|


#### ``genres`` table:

|genre_id (PK)|genre|
|----|----|
|1|Drama|
|2|Sci-Fi|
|3|Thriller|
|4|Romance|
|5|Western|
|6|Crime|
|...|...|


#### ``movie_genres`` table:

|movie_id (PK)|genre_id (PK)|
|----|----|
|1|1|
|1|2|
|1|3|
|2|1|
|2|4|
|2|5|
|3|6|
|3|1|
|...|...|

#### ``orders`` table:

|customer_id (PK)|movie_id (PK)|movie_price|purchase_datetime|
|----|----|----|----|
|1|1|4.04|2018-08-27 17:29:17|
|2|2|6.96|2018-08-20 08:42:38|
|3|3|13.47|2018-07-24 00:15:35|
|...|...|...|...|

### Entity Relationship Diagram

During the initial design phase and normalization, it is always good to draw the Entity Relationship Diagram (ERD) in order to visualize entities and relationship sets. This often helps to determine proper cardinalities and spot mistakes.

Below is the link to the ERD as defined after normalization.

https://lucid.app/lucidchart/282a41d2-2969-451e-98f1-edb476f8e4d5/view

## Create database tables

Based on the normalization plan we are now ready to create the database tables. Before running the commands below, make sure you have created a ``movie_store`` database in pgAdmin or psql.

In [8]:
# Load the PostgreSQL driver
drv <- dbDriver('PostgreSQL')

# Create a connection
con <- dbConnect(drv, dbname = 'movie_store',
                 host = 'localhost', port = 5432,
                 user = 'postgres', password = 'pwd4APAN5310')

# Pass the SQL statements that create all tables
stmt <- "
    CREATE TABLE customers (
        customer_id   integer,
        first_name    varchar(50) NOT NULL,
        last_name     varchar(50) NOT NULL,
        email         varchar(50) NOT NULL,
        cell_phone    varchar(20),
        PRIMARY KEY (customer_id)
    );
    
    CREATE TABLE movies (
        movie_id      integer,
        movie_title   varchar(100) NOT NULL,
        PRIMARY KEY (movie_id)
    );
    
    CREATE TABLE genres (
        genre_id      integer,
        genre         varchar(20) NOT NULL,
        PRIMARY KEY (genre_id)
    );
    
    CREATE TABLE movie_genres (
        movie_id      integer,
        genre_id      integer,
        PRIMARY KEY (movie_id, genre_id),
        FOREIGN KEY (movie_id) REFERENCES movies (movie_id),
        FOREIGN KEY (genre_id) REFERENCES genres (genre_id)
    );
    
    CREATE TABLE orders (
        customer_id   integer,
        movie_id      integer,
        movie_price   numeric(5,2) NOT NULL,
        purchase_datetime  timestamp NOT NULL,
        PRIMARY KEY (customer_id, movie_id),
        FOREIGN KEY (customer_id) REFERENCES customers (customer_id),
        FOREIGN KEY (movie_id) REFERENCES movies (movie_id)
    );
"

# Execute the statement to create tables
dbGetQuery(con, stmt)

<0 x 0 matrix>

## Extract, Transform and Load (ETL)

With the database and all tables created (3NF), it is now time to extract, transform and load (ETL) the dataset into the database. In order to do so we will have to perform several data transformations on the loaded dataframe, ``df`` in order to create all primary keys and maintain proper relationships. For hints, we may refer back to the normalization plan and the steps taken to produce a 3NF design.

Since all customers in the dataset are unique, we can create a new column with incrementing integer numbers for ``customer_id``.

In [9]:
df$customer_id <- 1:nrow(df)

Let's take a look:

In [10]:
head(df)

first_name,last_name,email,cell_phone,movie_title,movie_genres,purchase_datetime,movie_price,customer_id
Jackqueline,Dulanty,jdulanty0@goo.ne.jp,+960-657-528-2938,"Thirteenth Floor, The",Drama|Sci-Fi|Thriller,2018-08-27 17:29:17,4.04,1
Merrili,Fost,mfost1@nps.gov,+351-633-272-4106,"Furies, The",Drama|Romance|Western,2018-08-20 08:42:38,6.96,2
Shae,Haberfield,shaberfield2@ihg.com,+62-629-240-2964,Smart Money,Crime|Drama,2018-07-24 00:15:35,13.47,3
Kennie,Church,kchurch3@nature.com,+381-486-928-8686,Wishful Thinking,Comedy,2018-09-22 11:01:17,13.40,4
Shadow,Puckett,spuckett4@shareasale.com,+27-611-497-3264,You Are God (Jestes Bogiem),Drama,2018-01-15 08:41:31,10.38,5
Rici,Burnett,rburnett5@state.tx.us,,Through the Forest,Drama,2018-04-01 05:16:02,5.36,6


We can now easily create a subset of ``df`` corresponding to the ``customers`` database table.

In [11]:
customers_df <- df[c('customer_id', 'first_name', 'last_name', 'email', 'cell_phone')]

In [12]:
head(customers_df)

customer_id,first_name,last_name,email,cell_phone
1,Jackqueline,Dulanty,jdulanty0@goo.ne.jp,+960-657-528-2938
2,Merrili,Fost,mfost1@nps.gov,+351-633-272-4106
3,Shae,Haberfield,shaberfield2@ihg.com,+62-629-240-2964
4,Kennie,Church,kchurch3@nature.com,+381-486-928-8686
5,Shadow,Puckett,spuckett4@shareasale.com,+27-611-497-3264
6,Rici,Burnett,rburnett5@state.tx.us,


And it is now possible to load customer data to the database.

*Note: remember that the connection url, engine and connection were initialized earlier therefore we do not have to repeate them hereon.*

In [13]:
dbWriteTable(con, name="customers", value=customers_df, row.names=FALSE, append=TRUE)

[1] TRUE

One table down, four to go...

Next, we will work on the ``movies`` table. Within the dataset there are repeating movie titles so we cannot simply add a column with incrementing integer numbers for the primary key of movies as this would lead to movies with multiple primary keys. There are several approaches to tackle this problem. One could be to write a function or a for-loop that loops through movie titles and assigns integer numbers accordingly. Since this is a relatively simple dataset, we will create a temporary dataframe with the unique movie titles only, add a column of incrementing integer numbers and then map these numbers back to the main dataframe, ``df``.

In [14]:
# Create temporary dataframe with unique movie titles
temp_movie_df <- data.frame('movie_title' = unique(df$movie_title))

# Add incrementing integers
temp_movie_df$movie_id <- 1:nrow(temp_movie_df) 

In [15]:
head(temp_movie_df)

movie_title,movie_id
"Thirteenth Floor, The",1
"Furies, The",2
Smart Money,3
Wishful Thinking,4
You Are God (Jestes Bogiem),5
Through the Forest,6


Before adding the ``movie_id`` primary key to the main dataframe, ``df``, we can push the movie data to the database:

In [16]:
dbWriteTable(con, name="movies", value=temp_movie_df, row.names=FALSE, append=TRUE)

[1] TRUE

Now we can add a new column to the main dataframe, ``df``, for the primary key of movies, ``movie_id``. This involves two steps. First, using ``temp_movie_df`` to create a list mapping ``movie_id`` with each movie in ``df`` and then inserting this list to ``df`` as a new column.

In [17]:
# Map movie_id
movie_id_list <- sapply(df$movie_title, function(x) temp_movie_df$movie_id[temp_movie_df$movie_title == x])
                        
# Add movie_id to the main dataframe
df$movie_id <- movie_id_list

Let's check that a few repeating movies have the same primary key:

In [18]:
df[c('movie_id', 'movie_title', 'movie_price')][df$movie_title %in% c('Afterwards','Whirlygirl'),]

,movie_id,movie_title,movie_price
156,156,Whirlygirl,2.92
504,500,Afterwards,13.13
525,500,Afterwards,14.88
695,156,Whirlygirl,2.24
816,156,Whirlygirl,17.97


And a quick look into the expanded dataframe:

In [19]:
head(df)

first_name,last_name,email,cell_phone,movie_title,movie_genres,purchase_datetime,movie_price,customer_id,movie_id
Jackqueline,Dulanty,jdulanty0@goo.ne.jp,+960-657-528-2938,"Thirteenth Floor, The",Drama|Sci-Fi|Thriller,2018-08-27 17:29:17,4.04,1,1
Merrili,Fost,mfost1@nps.gov,+351-633-272-4106,"Furies, The",Drama|Romance|Western,2018-08-20 08:42:38,6.96,2,2
Shae,Haberfield,shaberfield2@ihg.com,+62-629-240-2964,Smart Money,Crime|Drama,2018-07-24 00:15:35,13.47,3,3
Kennie,Church,kchurch3@nature.com,+381-486-928-8686,Wishful Thinking,Comedy,2018-09-22 11:01:17,13.40,4,4
Shadow,Puckett,spuckett4@shareasale.com,+27-611-497-3264,You Are God (Jestes Bogiem),Drama,2018-01-15 08:41:31,10.38,5,5
Rici,Burnett,rburnett5@state.tx.us,,Through the Forest,Drama,2018-04-01 05:16:02,5.36,6,6


Next, we will work with movie genres. The main task is to split multiple genres in one cell. This should be done **row-wise**, meaning, a row with multiple genres will be expanded in multiple rows, each row holding the same data for all columns except for the genre column which will hold each genre. This operation is crucial since it will create the relationship between movies and genres.

This task requires the following steps:

In [20]:
# Split genres
s_genres <- strsplit(as.character(df$movie_genres), split = "|", fixed=TRUE)

# Create a new expanded dataframe
df <- data.frame(customer_id = rep(df$customer_id, sapply(s_genres, length)),
                 first_name = rep(df$first_name, sapply(s_genres, length)),
                 last_name = rep(df$last_name, sapply(s_genres, length)),
                 email = rep(df$email, sapply(s_genres, length)),
                 cell_phone = rep(df$cell_phone, sapply(s_genres, length)),
                 movie_id = rep(df$movie_id, sapply(s_genres, length)),
                 movie_title = rep(df$movie_title, sapply(s_genres, length)),
                 purchase_datetime = rep(df$purchase_datetime, sapply(s_genres, length)),
                 movie_price = rep(df$movie_price, sapply(s_genres, length)),
                 movie_genres = unlist(s_genres))

The main dataframe, ``df`` now looks like this:

In [21]:
head(df, 10)

customer_id,first_name,last_name,email,cell_phone,movie_id,movie_title,purchase_datetime,movie_price,movie_genres
1,Jackqueline,Dulanty,jdulanty0@goo.ne.jp,+960-657-528-2938,1,"Thirteenth Floor, The",2018-08-27 17:29:17,4.04,Drama
1,Jackqueline,Dulanty,jdulanty0@goo.ne.jp,+960-657-528-2938,1,"Thirteenth Floor, The",2018-08-27 17:29:17,4.04,Sci-Fi
1,Jackqueline,Dulanty,jdulanty0@goo.ne.jp,+960-657-528-2938,1,"Thirteenth Floor, The",2018-08-27 17:29:17,4.04,Thriller
2,Merrili,Fost,mfost1@nps.gov,+351-633-272-4106,2,"Furies, The",2018-08-20 08:42:38,6.96,Drama
2,Merrili,Fost,mfost1@nps.gov,+351-633-272-4106,2,"Furies, The",2018-08-20 08:42:38,6.96,Romance
2,Merrili,Fost,mfost1@nps.gov,+351-633-272-4106,2,"Furies, The",2018-08-20 08:42:38,6.96,Western
3,Shae,Haberfield,shaberfield2@ihg.com,+62-629-240-2964,3,Smart Money,2018-07-24 00:15:35,13.47,Crime
3,Shae,Haberfield,shaberfield2@ihg.com,+62-629-240-2964,3,Smart Money,2018-07-24 00:15:35,13.47,Drama
4,Kennie,Church,kchurch3@nature.com,+381-486-928-8686,4,Wishful Thinking,2018-09-22 11:01:17,13.40,Comedy
5,Shadow,Puckett,spuckett4@shareasale.com,+27-611-497-3264,5,You Are God (Jestes Bogiem),2018-01-15 08:41:31,10.38,Drama


Clearly, we now have genres that are repeating. In order to assign primary keys to the genres, we will follow the same process as with the movie titles.

In [22]:
# Create temporary dataframe with unique movie genres
temp_genre_df <- data.frame(genre = unique(df$movie_genres))

# Add incrementing integers for genre_id
temp_genre_df$genre_id <- 1:nrow(temp_genre_df) 

This temporary genre dataframe is:

In [23]:
temp_genre_df

genre,genre_id
Drama,1
Sci-Fi,2
Thriller,3
Romance,4
Western,5
Crime,6
Comedy,7
Action,8
Adventure,9
Animation,10


Before adding the ``genre_id`` primary key to the main dataframe, ``df``, we can push the genre data to the database:

In [24]:
dbWriteTable(con, name="genres", value=temp_genre_df, row.names=FALSE, append=TRUE)

[1] TRUE

Following the same process as with movie titles, we can now add a new column to the main dataframe, ``df``, for the primary key of genres, ``genre_id``. First, using ``temp_genre_df`` to create a list mapping ``genre_id`` with each genre in ``df`` and then inserting this list to ``df`` as a new column.

In [25]:
# Map genre_id
genre_id_list <- sapply(df$movie_genres, function(x) temp_genre_df$genre_id[temp_genre_df$genre == x])
                        
# Add genre_id to the main dataframe
df$genre_id <- genre_id_list

And ``df`` is now:

In [26]:
head(df, 10)

customer_id,first_name,last_name,email,cell_phone,movie_id,movie_title,purchase_datetime,movie_price,movie_genres,genre_id
1,Jackqueline,Dulanty,jdulanty0@goo.ne.jp,+960-657-528-2938,1,"Thirteenth Floor, The",2018-08-27 17:29:17,4.04,Drama,1
1,Jackqueline,Dulanty,jdulanty0@goo.ne.jp,+960-657-528-2938,1,"Thirteenth Floor, The",2018-08-27 17:29:17,4.04,Sci-Fi,2
1,Jackqueline,Dulanty,jdulanty0@goo.ne.jp,+960-657-528-2938,1,"Thirteenth Floor, The",2018-08-27 17:29:17,4.04,Thriller,3
2,Merrili,Fost,mfost1@nps.gov,+351-633-272-4106,2,"Furies, The",2018-08-20 08:42:38,6.96,Drama,1
2,Merrili,Fost,mfost1@nps.gov,+351-633-272-4106,2,"Furies, The",2018-08-20 08:42:38,6.96,Romance,4
2,Merrili,Fost,mfost1@nps.gov,+351-633-272-4106,2,"Furies, The",2018-08-20 08:42:38,6.96,Western,5
3,Shae,Haberfield,shaberfield2@ihg.com,+62-629-240-2964,3,Smart Money,2018-07-24 00:15:35,13.47,Crime,6
3,Shae,Haberfield,shaberfield2@ihg.com,+62-629-240-2964,3,Smart Money,2018-07-24 00:15:35,13.47,Drama,1
4,Kennie,Church,kchurch3@nature.com,+381-486-928-8686,4,Wishful Thinking,2018-09-22 11:01:17,13.40,Comedy,7
5,Shadow,Puckett,spuckett4@shareasale.com,+27-611-497-3264,5,You Are God (Jestes Bogiem),2018-01-15 08:41:31,10.38,Drama,1


We do not need to assign any other primary keys. For the remaining two tables in our schema, we must only slice the main dataframe, eliminate duplicate rows and push to the database.

Let's focus on ``movie_genres``.

In [27]:
head(df[c('movie_id', 'genre_id')], 20)

movie_id,genre_id
1,1
1,2
1,3
2,1
2,4
2,5
3,6
3,1
4,7
5,1


In order to check if there are any duplicate rows we should look into the rows for repeating movies, for example, "Magic Man".

In [28]:
df[df$movie_title == 'Magic Man',]

,customer_id,first_name,last_name,email,cell_phone,movie_id,movie_title,purchase_datetime,movie_price,movie_genres,genre_id
392,193,Johnny,Fullerlove,jfullerlove5c@economist.com,+351-495-428-5920,193,Magic Man,2018-06-27 16:56:51,14.77,Crime,6
393,193,Johnny,Fullerlove,jfullerlove5c@economist.com,+351-495-428-5920,193,Magic Man,2018-06-27 16:56:51,14.77,Mystery,16
394,193,Johnny,Fullerlove,jfullerlove5c@economist.com,+351-495-428-5920,193,Magic Man,2018-06-27 16:56:51,14.77,Thriller,3
442,222,Lauree,Dingwall,ldingwall65@springer.com,+212-807-564-8447,193,Magic Man,2017-06-09 07:07:50,13.73,Crime,6
443,222,Lauree,Dingwall,ldingwall65@springer.com,+212-807-564-8447,193,Magic Man,2017-06-09 07:07:50,13.73,Mystery,16
444,222,Lauree,Dingwall,ldingwall65@springer.com,+212-807-564-8447,193,Magic Man,2017-06-09 07:07:50,13.73,Thriller,3


For table ``movie_genres``, the composite primary key is made up from ``movie_id`` and ``genre_id``. As is made obvious above, pushing this ``df`` slide to the database without removinf duplicates would return an error since both Johny and Lauree purchased "Magic Man" (movie_id = 193 with gernre_id 6, 16, 3).

Dropping duplicate rows and pushing to the database can be done in one command:

In [29]:
dbWriteTable(con, name="movie_genres", 
             value=df[c('movie_id', 'genre_id')][!duplicated(df[c('movie_id', 'genre_id')]),], 
             row.names=FALSE, append=TRUE)

[1] TRUE

Finally, the process is the same for the ``orders`` table. If we look at the slice:

In [30]:
head(df[c('customer_id', 'movie_id', 'movie_price', 'purchase_datetime')])

customer_id,movie_id,movie_price,purchase_datetime
1,1,4.04,2018-08-27 17:29:17
1,1,4.04,2018-08-27 17:29:17
1,1,4.04,2018-08-27 17:29:17
2,2,6.96,2018-08-20 08:42:38
2,2,6.96,2018-08-20 08:42:38
2,2,6.96,2018-08-20 08:42:38


There are clearly duplicate rows that need to be removed before pushing to the database.

In [31]:
head(df[c('customer_id', 'movie_id', 'movie_price', 'purchase_datetime')][
    !duplicated(df[c('customer_id', 'movie_id', 'movie_price', 'purchase_datetime')]),])

,customer_id,movie_id,movie_price,purchase_datetime
1,1,1,4.04,2018-08-27 17:29:17
4,2,2,6.96,2018-08-20 08:42:38
7,3,3,13.47,2018-07-24 00:15:35
9,4,4,13.40,2018-09-22 11:01:17
10,5,5,10.38,2018-01-15 08:41:31
11,6,6,5.36,2018-04-01 05:16:02


Which we can also do in one command:

In [32]:
dbWriteTable(con, name="orders", 
             value=df[c('customer_id', 'movie_id', 'movie_price', 'purchase_datetime')][
                 !duplicated(df[c('customer_id', 'movie_id', 'movie_price', 'purchase_datetime')]),], 
             row.names=FALSE, append=TRUE)

[1] TRUE

## Spot Checks & Validation

With all database tables populated, we must run some tests to make sure that data was tranferred properly and all relations have been upheld.

Let's take a quick look at the original dataset:

In [33]:
df2 <- read.csv('hw8_dataset_f18.csv')
head(df2)

first_name,last_name,email,cell_phone,movie_title,movie_genres,purchase_datetime,movie_price
Jackqueline,Dulanty,jdulanty0@goo.ne.jp,+960-657-528-2938,"Thirteenth Floor, The",Drama|Sci-Fi|Thriller,2018-08-27 17:29:17,4.04
Merrili,Fost,mfost1@nps.gov,+351-633-272-4106,"Furies, The",Drama|Romance|Western,2018-08-20 08:42:38,6.96
Shae,Haberfield,shaberfield2@ihg.com,+62-629-240-2964,Smart Money,Crime|Drama,2018-07-24 00:15:35,13.47
Kennie,Church,kchurch3@nature.com,+381-486-928-8686,Wishful Thinking,Comedy,2018-09-22 11:01:17,13.40
Shadow,Puckett,spuckett4@shareasale.com,+27-611-497-3264,You Are God (Jestes Bogiem),Drama,2018-01-15 08:41:31,10.38
Rici,Burnett,rburnett5@state.tx.us,,Through the Forest,Drama,2018-04-01 05:16:02,5.36


We will query for, say, the genres of the movie "The Furies".

*Note: remember that the connection url, engine and connection were initialized earlier therefore we do not have to repeate them hereon.*

In [34]:
# Pass the SQL statement to filter data 
stmt <- "

    SELECT g.genre 
    FROM genres g
    JOIN movie_genres mg ON g.genre_id = mg.genre_id
    JOIN movies m ON m.movie_id = mg.movie_id
    WHERE m.movie_title = 'Furies, The';

"

# Execute the statement and store results in a temporary dataframe
temp_df <- dbGetQuery(con, stmt)

# Show results
temp_df

genre
Drama
Romance
Western


So far so good. It would be wise to test the same query on movie titles that were repeating, such as "Magic Man" to make sure that genres for this movie have not been stored more than once.

In [35]:
df2[df2$movie_title == 'Magic Man',]

,first_name,last_name,email,cell_phone,movie_title,movie_genres,purchase_datetime,movie_price
193,Johnny,Fullerlove,jfullerlove5c@economist.com,+351-495-428-5920,Magic Man,Crime|Mystery|Thriller,2018-06-27 16:56:51,14.77
222,Lauree,Dingwall,ldingwall65@springer.com,+212-807-564-8447,Magic Man,Crime|Mystery|Thriller,2017-06-09 07:07:50,13.73


In [36]:
# Pass the SQL statement to filter data 
stmt <- "

    SELECT g.genre 
    FROM genres g
    JOIN movie_genres mg ON g.genre_id = mg.genre_id
    JOIN movies m ON m.movie_id = mg.movie_id
    WHERE m.movie_title = 'Magic Man';

"

# Execute the statement and store results in a temporary dataframe
temp_df <- dbGetQuery(con, stmt)

# Show results
temp_df

genre
Crime
Mystery
Thriller


Great! Let's get a quick count of all movies that have been purchased more than once:

In [37]:
# Pass the SQL statement to filter data 
stmt <- "

    SELECT m.movie_title, COUNT(m.movie_title) AS movie_count
    FROM movies m
    JOIN orders o ON o.movie_id = m.movie_id
    GROUP BY m.movie_title
    HAVING COUNT(m.movie_title) > 1
    ORDER BY movie_count DESC;

"

# Execute the statement and store results in a temporary dataframe
temp_df <- dbGetQuery(con, stmt)

# Show results
temp_df

movie_title,movie_count
Whirlygirl,3
Only Old Men Are Going to Battle (V boy idut odni stariki),2
The Beehive,2
Invisible Agent,2
Strike (Stachka),2
I Saw the Sun (Günesi gördüm),2
Afterwards,2
"Borrowers, The",2
Burn After Reading,2
Slap Her... She's French (a.k.a. She Gets What She Wants),2


It appears that the most popular movie was "Whirlygirl". Similarly, we can get counts of movie genres:

In [38]:
# Pass the SQL statement to filter data 
stmt <- "

    SELECT g.genre, COUNT(g.genre) AS genre_count
    FROM genres g
    JOIN movie_genres mg ON g.genre_id = mg.genre_id
    GROUP BY g.genre
    ORDER BY genre_count DESC;

"

# Execute the statement and store results in a temporary dataframe
temp_df <- dbGetQuery(con, stmt)

# Show results
temp_df

genre,genre_count
Drama,479
Comedy,293
Thriller,167
Romance,131
Action,125
Crime,104
Horror,103
Adventure,81
Documentary,79
Mystery,55


Finally, since "Whirlygirl" was sold more than once, let's look at when it was purchased and by whom:

In [39]:
# Pass the SQL statement to filter data 
stmt <- "

    SELECT c.first_name, c.last_name, m. movie_title, o.movie_price, o.purchase_datetime
    FROM orders o
    JOIN movies m ON m.movie_id = o.movie_id
    JOIN customers c ON c.customer_id = o.customer_id
    WHERE m.movie_title = 'Whirlygirl';

"

# Execute the statement and store results in a temporary dataframe
temp_df <- dbGetQuery(con, stmt)

# Show results
temp_df

first_name,last_name,movie_title,movie_price,purchase_datetime
Madel,Kirkland,Whirlygirl,2.92,2017-06-04 08:21:41
Elwyn,Furness,Whirlygirl,2.24,2017-02-05 21:26:18
Eyde,Lippitt,Whirlygirl,17.97,2018-04-29 07:37:39


The result matching the original dataset for the same movie:

In [40]:
df2[df2$movie_title == 'Whirlygirl',]

,first_name,last_name,email,cell_phone,movie_title,movie_genres,purchase_datetime,movie_price
156,Madel,Kirkland,mkirkland4b@arstechnica.com,+63-712-138-5062,Whirlygirl,Drama|Romance,2017-06-04 08:21:41,2.92
695,Elwyn,Furness,efurnessja@xrea.com,+86-618-650-7587,Whirlygirl,Drama|Romance,2017-02-05 21:26:18,2.24
816,Eyde,Lippitt,elippittmn@virginia.edu,+86-120-329-8613,Whirlygirl,Drama|Romance,2018-04-29 07:37:39,17.97


## Conclusion

This exercise was a basic normalization and ETL process aimed at building your skills in developing and populating relational databases. A small change in the original dataset could lead to a completely different database design and ETL process. It is therefore very important to develop your coding skills so that you are comfortable and flexible implementing different approaches. There is no effective book or guide on how to implement ETL processes. It all comes down to creativity and lots of practice.